In [1]:
import pandas as pd
import gc
rcc_train = pd.read_csv("../data/rcc_train.csv")
y_train = pd.read_csv("../data/y_train.csv", index_col="key_value")
rcc_test = pd.read_csv("../data/rcc_test.csv")

In [2]:
rcc_train.shape, rcc_test.shape

((30391626, 9), (34351754, 9))

In [ ]:
### se puede observar que en el rcc_train aquellos clientes que tienen un solo registro, es porque se refiere al ultimo
### periodo de la data de train que es del febrero del 2018, cuyo RIESGO_DIRECTO es 1 y COD_CLASIFICACION_DEUDOR es 0 
### su condicion (días de atraso) es 0 siendo estos cerca de 623 clientes. Pero OJO el 9% de los nuevos clientes tienen un target igual a 1

### el target de la data train representa el score crediticio al febrero del 2018 
### por lo que se sobre-entiende que lo que se quiere predecir es el score crediticio al febrero del 2019

### La data train abarca desde 201703 hasta 201802
### La data test comprende desde 201803 hasta 201902
### Los primeros registros de los clientes tienen un RIESGO DIRECTO de 1 ***

### La distribucion del target general es de 16% para 1 y 84% para los 0
### A mayor COD_CLASIFICACION_DEUDOR y codmes mes cercano, el target se aproxima a 1. Ejm COD_CLASIFICACION_DEUDOR=5 y codmes=201802
### el target esta representado por el 97% de 1 y 3% de cero 

In [ ]:
### condicion, saldo   ..... son variables cuantitativas
### tipo_credito en train no tiene la clase de 7 como en el test, pero este tiene solo 1 valor (8 clases) ### Frecuencia 11
### cod_instit_financiera , 12 clases más se encuentran en el train que en el test (mas de 100 clases) ### Frecuencia 34 ----
### PRODUCTO, el train tiene la clase 255 y el test no (42 clases) ### Frecuencia 1  ----
### RIESGO_DIRECTO es lo mismo , es lo mismo en el test y en el train (5 clases) ### Frecuencia -1
### COD_CLASIFICACION_DEUDOR , es lo mismo en el test y en el train (6 clases) ### Frecuencia 0
### en el rcc_test, faltan valores en la columma de cod_instit_financiera y producto

In [4]:
bins = [-1, 0, 10, 20, 30, 60, 90, 180, 360, 720, float("inf")]
rcc_train["condicion"] = pd.cut(rcc_train.condicion, bins, labels = range(0,10))
rcc_test["condicion"] = pd.cut(rcc_test.condicion, bins, labels = range(0,10))
rcc_test['cod_instit_financiera'] = rcc_test['cod_instit_financiera'].fillna(34)
rcc_test['PRODUCTO'] = rcc_test['PRODUCTO'].fillna(1)

In [5]:
rcc_train = rcc_train.astype({'condicion':'int32',
                  'tipo_credito':'int32',
                  'cod_instit_financiera':'int32',
                  'PRODUCTO':'int32',
                  'RIESGO_DIRECTO':'int32',
                  'COD_CLASIFICACION_DEUDOR':'int32'})
rcc_test = rcc_test.astype({'condicion':'int32',
                  'tipo_credito':'int32',
                  'cod_instit_financiera':'int32',
                  'PRODUCTO':'int32',
                  'RIESGO_DIRECTO':'int32',
                  'COD_CLASIFICACION_DEUDOR':'int32'})

In [6]:
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda

In [7]:
from tqdm.notebook import tqdm 
list_rcc_train_agg = []

moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_rcc = {
           'cod_instit_financiera':['nunique',moda],
            'COD_CLASIFICACION_DEUDOR':['nunique','max',moda]
          }
for n,i in enumerate(tqdm(sorted(set(rcc_train.codmes),reverse=True))):
    if i in [201802,201712,201709,201703]:
        for c in ['tipo_credito','cod_instit_financiera','PRODUCTO','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR','condicion']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()
    print(f'haciendo agg ')
    rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby('key_value').agg(agg_rcc)
    rcc_train_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_train_agg.columns]
    list_rcc_train_agg.append(rcc_train_agg)
    gc.collect()
        
rcc_train_ = pd.concat(list_rcc_train_agg, axis=1)
del rcc_train, list_rcc_train_agg

haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo condicion
haciendo agg 
haciendo agg 
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo condicion
haciendo agg 
haciendo agg 
haciendo agg 
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo condicion
haciendo agg 
haciendo agg 
haciendo agg 
haciendo agg 
haciendo agg 
haciendo agg 
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo condicion
haciendo agg 



In [8]:
rcc_train_.shape

(358487, 757)

In [9]:
from tqdm.notebook import tqdm 
list_rcc_test_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_test.codmes),reverse=True))):
    if i in [201902,201812,201809,201803]:
        for c in ['tipo_credito','cod_instit_financiera','PRODUCTO','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR','condicion']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
    print(f'haciendo agg')    
    rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby('key_value').agg(agg_rcc)
    rcc_test_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_test_agg.columns]
    list_rcc_test_agg.append(rcc_test_agg)
    gc.collect()
    
rcc_test_ = pd.concat(list_rcc_test_agg, axis=1)
del rcc_test, list_rcc_test_agg

haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo condicion
haciendo agg
haciendo agg
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo condicion
haciendo agg
haciendo agg
haciendo agg
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo condicion
haciendo agg
haciendo agg
haciendo agg
haciendo agg
haciendo agg
haciendo agg
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo condicion
haciendo agg



In [10]:
rcc_train_.shape, rcc_test_.shape

((358487, 757), (396666, 709))

In [11]:
print(rcc_train_.shape)
keep_cols = list(set(rcc_train_.columns).intersection(set(rcc_test_.columns)))
rcc_train_ = rcc_train_[keep_cols].copy()
rcc_test_ = rcc_test_[keep_cols].copy()
print(rcc_train_.shape)

(358487, 757)
(358487, 702)


In [12]:
rcc_train_.shape, rcc_test_.shape

((358487, 702), (396666, 702))

In [13]:
(rcc_train_.isna().sum().sort_values(ascending = False)/rcc_train_.shape[0]).head(25)

cod_instit_financiera_mode_ult1mes        0.175111
cod_instit_financiera_mode_ult2mes        0.103094
cod_instit_financiera_mode_ult3mes        0.074781
cod_instit_financiera_mode_ult4mes        0.057930
cod_instit_financiera_mode_ult5mes        0.045483
cod_instit_financiera_mode_ult6mes        0.040197
cod_instit_financiera_mode_ult7mes        0.033418
cod_instit_financiera_mode_ult8mes        0.030160
cod_instit_financiera_mode_ult9mes        0.026419
cod_instit_financiera_mode_ult10mes       0.023780
cod_instit_financiera_mode_ult12mes       0.021616
cod_instit_financiera_mode_ult11mes       0.021541
COD_CLASIFICACION_DEUDOR_mode_ult1mes     0.007308
COD_CLASIFICACION_DEUDOR_mode_ult6mes     0.006357
COD_CLASIFICACION_DEUDOR_mode_ult2mes     0.006028
COD_CLASIFICACION_DEUDOR_mode_ult9mes     0.004709
COD_CLASIFICACION_DEUDOR_mode_ult3mes     0.004293
COD_CLASIFICACION_DEUDOR_mode_ult4mes     0.002817
COD_CLASIFICACION_DEUDOR_mode_ult12mes    0.002737
COD_CLASIFICACION_DEUDOR_mode_u

In [14]:
#Función para obtener variables con alta correlación
def get_correlated(data, cut=0.85):
    correlated_features = set()
    correlation_matrix = data.corr()
    
    for i in range(len(correlation_matrix .columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > cut:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
            
    return correlated_features
print(rcc_train_.shape, rcc_test_.shape)
correlated = get_correlated(rcc_train_,cut=0.95)
rcc_train_ = rcc_train_.drop(columns=correlated)
rcc_test_ = rcc_test_.drop(columns=correlated)
print(rcc_train_.shape, rcc_test_.shape)

(358487, 702) (396666, 702)
(358487, 346) (396666, 346)


In [15]:
rcc_train_.to_csv('../data/intermediate/rcc_train.csv')
rcc_test_.to_csv('../data/intermediate/rcc_test.csv')